In [40]:
import pandas as pd
import os 
import pulp

# read data


In [2]:
os.getcwd()

'c:\\Users\\WS GARCIA\\Projects\\Personnel_Scheduling_1_stage\\model'

In [49]:
demanda=pd.read_excel('c:\\Users\\WS GARCIA\\Projects\\Personnel_Scheduling_1_stage\\data\\Dataton 2023 Etapa 1.xlsx',sheet_name='demand')
workers=pd.read_excel('c:\\Users\\WS GARCIA\\Projects\\Personnel_Scheduling_1_stage\\data\\Dataton 2023 Etapa 1.xlsx',sheet_name='workers')

In [50]:
demanda=demanda.loc[:,['fecha_hora','demanda']].set_index('fecha_hora')

In [47]:
# Crear un DataFrame de demanda
data = {'Tiempo': pd.date_range(start='7:30', end='18:45', freq='15T').astype(str),
        'Demanda': demanda.demanda.tolist()}

df_demanda = pd.DataFrame(data)
df_demanda['Tiempo'] = pd.to_datetime(df_demanda['Tiempo'])

# Total de trabajadores disponibles
total_trabajadores = 8

# Crea una instancia del problema
problem = pulp.LpProblem("ProgramacionPersonal", pulp.LpMinimize)

# Variables de decisión
# x[empleado][tiempo] representa si el empleado trabaja en un determinado momento
empleados = list(range(total_trabajadores))
tiempos = df_demanda['Tiempo']
# Estados posibles
estados = ["Nada","Trabaja", "Descansa", "PausaActiva", "Almuerzo"]

x = pulp.LpVariable.dicts('Estado', (empleados, tiempos), 0, len(estados) - 1, pulp.LpInteger)


In [46]:
x

{0: {Timestamp('2023-10-19 07:30:00'): Estado_0_2023_10_19_07:30:00,
  Timestamp('2023-10-19 07:45:00'): Estado_0_2023_10_19_07:45:00,
  Timestamp('2023-10-19 08:00:00'): Estado_0_2023_10_19_08:00:00,
  Timestamp('2023-10-19 08:15:00'): Estado_0_2023_10_19_08:15:00,
  Timestamp('2023-10-19 08:30:00'): Estado_0_2023_10_19_08:30:00,
  Timestamp('2023-10-19 08:45:00'): Estado_0_2023_10_19_08:45:00,
  Timestamp('2023-10-19 09:00:00'): Estado_0_2023_10_19_09:00:00,
  Timestamp('2023-10-19 09:15:00'): Estado_0_2023_10_19_09:15:00,
  Timestamp('2023-10-19 09:30:00'): Estado_0_2023_10_19_09:30:00,
  Timestamp('2023-10-19 09:45:00'): Estado_0_2023_10_19_09:45:00,
  Timestamp('2023-10-19 10:00:00'): Estado_0_2023_10_19_10:00:00,
  Timestamp('2023-10-19 10:15:00'): Estado_0_2023_10_19_10:15:00,
  Timestamp('2023-10-19 10:30:00'): Estado_0_2023_10_19_10:30:00,
  Timestamp('2023-10-19 10:45:00'): Estado_0_2023_10_19_10:45:00,
  Timestamp('2023-10-19 11:00:00'): Estado_0_2023_10_19_11:00:00,
  Times

In [39]:


# Función objetivo
# Minimizar la diferencia entre la demanda y la cantidad de trabajadores programados
problem += pulp.lpSum(df_demanda['Demanda'][i] - pulp.lpSum(x[empleado][tiempos[i]] for empleado in empleados) for i in range(len(df_demanda))), "MinimizarDemanda"

# Restricciones
for empleado in empleados:
    # Restricción: Cada empleado debe cumplir 8 horas de trabajo
    problem += pulp.lpSum(x[empleado][tiempo] for tiempo in tiempos) == 8

    # Restricciones 2 y 3: Mínimo 1 hora continua y máximo 2 horas continuas sin pausa activa
    for i in range(len(tiempos) - 3):
        problem += pulp.lpSum(x[empleado][tiempos[i]:tiempos[i + 3]]) >= 1
        problem += pulp.lpSum(x[empleado][tiempos[i]:tiempos[i + 3]]) <= 2

    # Restricción 4: Tiempo de almuerzo de 1 hora y media
    problem += pulp.lpSum(x[empleado][tiempos[tiempos.index('11:30'):tiempos.index('13:00')]]) == 3

    # Restricción 5: Hora mínima y máxima de almuerzo
    problem += x[empleado][tiempos.index('11:15')] == 0
    problem += x[empleado][tiempos.index('13:15')] == 0

    # Restricciones 6 y 7: Jornada laboral de 8 horas y horario continuo
    problem += pulp.lpSum(x[empleado][tiempo] for tiempo in tiempos) >= 8
    problem += x[empleado][tiempos[0]] == 0
    problem += x[empleado][tiempos[-1]] == 0

# Restricción 8: El último estado de la jornada laboral debe ser Trabaja
for empleado in empleados:
    problem += x[empleado][tiempos[-1]] == 1

# Restricción 9: Al menos 1 empleado en el estado Trabaja en cada franja horaria
for tiempo in tiempos:
    problem += pulp.lpSum(x[empleado][tiempo] for empleado in empleados) >= 1

# Restricción 10: Cualquier franja de trabajo debe durar entre 1 y 2 horas
for i in range(len(tiempos) - 1):
    problem += pulp.lpSum(x[empleado][tiempos[i]:tiempos[i + 1]]) >= 1
    problem += pulp.lpSum(x[empleado][tiempos[i]:tiempos[i + 1]]) <= 2

# Resuelve el problema
problem.solve()

# Imprime el resultado
print("Estado:", pulp.LpStatus[problem.status])

for empleado in empleados:
    trabajando = []
    for tiempo in tiempos:
        if pulp.value(x[empleado][tiempo]) == 1:
            trabajando.append(tiempo)
    print(f"Empleado {empleado}: {trabajando}")

print("Diferencia entre Demanda y Trabajadores Programados: ", pulp.value(problem.objective))




KeyError: slice(Timestamp('2023-10-19 07:30:00'), Timestamp('2023-10-19 08:15:00'), None)